<a href="https://colab.research.google.com/github/AndersonZangelmi/Exercicio_Python/blob/master/BD_API_GOV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run below commands in google colab
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.0
!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
# unzip it
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
# install findspark
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
import pyspark
print(pyspark.__version__)

3.0.1


In [20]:
!curl -X GET --header 'Accept: application/json' --header 'chave-api-dados: ff9c269c9bf8969335b8bfc15580e4d9' 'http://www.portaltransparencia.gov.br/api-de-dados/cartoes?mesExtratoInicio=01%2F2020&mesExtratoFim=07%2F2020&pagina=1' > consulta_portal_transparencia.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 26676    0 26676    0     0   7044      0 --:--:--  0:00:03 --:--:--  7042


In [21]:
#Instancia o contexto do Spark
sc = spark.sparkContext
#Realiza a leitura da nossa consulta realizada na API do Governo Federal sobre as Viagens
path = "/content/consulta_portal_transparencia.json"
#Declara dataframe df com o conteudo da api que estava no JSON
df = spark.read.json(path)

In [22]:
#Imprime a estrutura de dados do nosso dataframe
df.printSchema()
#Imprime uma amosta de dados ( para especificar as linhas basta colocar a quantidade entre ())
df.show()

root
 |-- dataTransacao: string (nullable = true)
 |-- estabelecimento: struct (nullable = true)
 |    |-- cnae: struct (nullable = true)
 |    |    |-- classe: string (nullable = true)
 |    |    |-- codigoClasse: string (nullable = true)
 |    |    |-- codigoDivisao: string (nullable = true)
 |    |    |-- codigoGrupo: string (nullable = true)
 |    |    |-- codigoSecao: string (nullable = true)
 |    |    |-- codigoSubclasse: string (nullable = true)
 |    |    |-- divisao: string (nullable = true)
 |    |    |-- grupo: string (nullable = true)
 |    |    |-- secao: string (nullable = true)
 |    |    |-- subclasse: string (nullable = true)
 |    |-- codigoFormatado: string (nullable = true)
 |    |-- complementoEndereco: string (nullable = true)
 |    |-- dataAbertura: string (nullable = true)
 |    |-- descricaoLogradouro: string (nullable = true)
 |    |-- enderecoEletronico: string (nullable = true)
 |    |-- localidadePessoa: string (nullable = true)
 |    |-- municipio: struct

In [17]:
dfPrepared = df.selectExpr("dataTransacao","estabelecimento.razaoSocialReceita","estabelecimento.nome","estabelecimento.municipio.uf.sigla","portador.nome as nome_do_portador","tipoCartao.descricao","valorTransacao")
dfPrepared.show()

+-------------+------------------+----+-----+----------------+---------+--------------+
|dataTransacao|razaoSocialReceita|nome|sigla|nome_do_portador|descricao|valorTransacao|
+-------------+------------------+----+-----+----------------+---------+--------------+
|         null|              null|null| null|            null|     null|          null|
+-------------+------------------+----+-----+----------------+---------+--------------+



In [18]:
#Declara tabela temporaria no SparkSQL
dfPrepared.createOrReplaceTempView("dfPrepared")

In [19]:
topTravels = spark.sql("select count(*),nome_do_portador as politico from dfPrepared group by 2 order by 1 desc")
topTravels.show()

+--------+--------+
|count(1)|politico|
+--------+--------+
|       1|    null|
+--------+--------+

